In [1]:
import matplotlib.pyplot as plt
from matplotlib import animation
from math import cos, sin, pi, radians, sqrt, exp
import numpy as np
import random as rd
import numpy as np

In [54]:
INNO = 0
def get_innovation():
    global INNO
    INNO += 1
    return INNO

MEMORY = []

NEURON_IDS = {}
def get_neuron_id(depth):
    global NEURON_IDS
    if depth not in NEURON_IDS:
        NEURON_IDS[depth]=[]
    ID = 'h' + str(depth) + '_' + str(len(NEURON_IDS[depth]))
    NEURON_IDS[depth].append(ID)
    return ID

def reset():
    global INNO, MEMORY, NEURON_IDS
    INNO, MEMORY, NEURON_IDS = 0, [], {}

In [3]:
class Brain_Gene():
    def __init__(self, _from=None, _to=None, innov=None, weight=None):
        global MEMORY
        if innov is None:
            self.innov = get_innovation()
            MEMORY.append(self)
        else:
            self.innov = innov
            
        self._from=_from
        self._to = _to
        
        if weight is None:
            self.weight = rd.random()
        else:
            self.weight = weight
            
        self.activated = True
    
    def __str__(self):
        s= str(self.innov) + ' : '
        s += str(self._from) + ' > ' + str(self._to)
        return s
        
    def copy(self):
        new = Brain_Gene(_from=self._from, _to=self._to, innov = self.innov, weight=self.weight)
        new.activated = self.activated
        return new
        
    def mutate_weight(self, std=0.1):
        self.weight += np.random.normal(scale=std)
        return self

In [4]:
class Neuron():
    def __init__(self, ID=None, depth=None):
        if ID is None:
            self.id = get_neuron_id(depth)
        else:
            self.id=ID
        self.sum=0
        self.funct = lambda x: 1 / (1 + exp(-x))
        
    def value(self):
        return self.funct(self.sum)
        

In [124]:
class Brain():
    def __init__(self, in_size, out_size=1, hidden_depth=0):
        self.in_size = in_size
        self.out_size = out_size
        self.hidden_depth = hidden_depth
        
        self.genome=[]
        
        self.neurons={
            'Input':{},
            'Output':{}
        }
        
        self.links={
            'Input':[],
            'Output':[]
        }
        
    def __str__(self):
        s = str(self.in_size) + ' | ' + str(self.hidden_depth) + ' | ' + str(self.out_size) + '\n'
        for l in self.neurons:
            s += l + ': ' + str(list(self.neurons[l].keys())) + '\n'
        s += '\n Links:\n'
        for l in self.links:
            for i in self.links[l]:
                s += str(i) + '\n'
        s += '\n'
        return s
    
    def __len__(self):
        return len(self.genome)
        
    def grow_brain(self, verbose=False):
        self.neurons={
            'Input':{},
            'Output':{}
        }
        
        self.links={
            'Input':[],
            'Output':[]
        }
        
        n = []
        
        for i in range(self.in_size):
            self.neurons['Input']['i_' + str(i)] = Neuron(ID='i_' + str(i))
            n.append('i_' + str(i))
            
        for i in range(self.out_size):
            self.neurons['Output']['o_' + str(i)] = Neuron(ID='o_' + str(i))
            n.append('o_' + str(i))
          
            
        # Get Neurons to create
        for g in self.genome:
            for x in [g._from, g._to]:
                if x not in n:
                    n.append(x)
                    if x[0]=='h':
                        self.hidden_depth = max(max(self.hidden_depth, int(x[1])+1), 1)
        
        for h in range(self.hidden_depth):
            self.neurons['h'+str(h)]={}
            self.links['h'+str(h)]=[]
            
        if verbose:
            print('\nNeurons', self.neurons, '\n')
            print('Links', self.links, '\n')
            
        # Classify links by execution order
        for g in self.genome:
            if g._to[0]=='o':
                self.links['Output'].append(g)
            elif g._to[0]=='h':
                self.links[g._to[:2]].append(g)
        
        for x in n:
            if x[0] =='h':
                self.neurons[x[:2]][x] = Neuron(ID=x)
                
        return self        
            
        
    def get_neuron(self, ID):
        if ID[0]=='i':
            return self.neurons['Input'][ID]
        elif ID[0]=='o':
            return self.neurons['Output'][ID]
        else:
            return self.neurons[ID[:2]][ID]
    
    def compute(self, v, verbose=False):
        assert len(v)==self.in_size
        
        if len(self.neurons) == 0:
            self.grow_brain()
            
        # Set input values
        for i in range(len(v)):
            ID = 'i' + str(i)
            if ID in self.neurons:
                self.neurons['Input'][ID].sum = v[i]
                if verbose: print('Input', ID)
        
        # Set other values, row by row
        for layer in self.links:
            if verbose: print('> Layer', layer)
            
            if layer != 'Input':
                # Reset values for the layer
                for n in self.neurons[layer]:
                    self.get_neuron(n).sum = 0
                    if verbose: print('Zeroed', n)
                    
            # Compute new sums with each link that goes to this layer
            for g in self.links[layer]:
                self.get_neuron(g._to).sum += self.get_neuron(g._from).value()
                if verbose: print('Done link', g)
                    
        output = []
        for o in self.neurons['Output']:
            output.append(self.get_neuron(o).value())
        
        return output
            
    def add_connection(self, _from=None, _to=None, weight=1):
        if len(self.neurons) == 0:
            self.grow_brain()
            
        if _from is None:
            layers = list(self.neurons.keys())
            layers.append(layers.pop(layers.index('Output')))
            neur = [self.get_neuron(i).id for l in layers for i in self.neurons[l]]
            avail = neur[:-self.out_size]
            if len(avail)>1:
                _from = avail[rd.randint(0, len(avail)-1)]
            elif len(avail)==0:
                return self
            else:
                _from = avail[0]
        
        if _to is None:
            index = max(self.in_size, neur.index(_from) + 1)
            avail = neur[index:]
            if len(avail)>1:
                _to = avail[rd.randint(0, len(avail)-1)]
            elif len(avail)==0:
                return self
            else:
                _to = avail[0]
        
        new_gene= Brain_Gene(_from=_from, _to=_to)
        self.genome.append(new_gene)
        self.grow_brain()
        return self
        
    def add_neuron(self, _from=None, _to=None):
        if len(self.genome)==0:
            return self
        if _from is None or _to is None:
            if len(self.genome)>1:
                link = self.genome[rd.randint(0, len(self.genome)-1)]
            else:
                link=self.genome[0]
        else:
            for l in self.genome:
                if l._from == _from and l._to == _to:
                    link=l
                    break
        _from, _to = link._from, link._to
        w = link.weight
        
        dist = self.get_depth_between(_from, _to)
        
        if _to[0] == 'o':
            new_id = get_neuron_id(depth=len(self.neurons)-2)
        
        elif dist >1:
            if _from[0] == 'h':
                new_id = get_neuron_id(depth= int(_from[1]) + 1)
            elif _to[0] == 'h':
                new_id = get_neuron_id(depth= int(_to[1]) -1)
    
        elif _from[0] == 'h':
            new_id = get_neuron_id(depth= int(_from[1]))   
            
        elif _from[0] == 'i':
            new_id = get_neuron_id(depth= 1)   
            
        self.add_connection(_from=_from, _to=new_id, weight=w)
        self.add_connection(_from=new_id, _to=_to)
        self.genome.pop(self.genome.index(link))
            
        self.grow_brain()
        return self
            
        
    def get_depth_between(self, a, b):
        d_a, d_b = -1, -1
        if a[0]=='o':
            d_a = self.hidden_depth + 1
        if b[0]=='o':
            d_b = self.hidden_depth + 1
        if a[0]=='h':
            d_a = int(a[1])
        if b[0]=='h':
            d_b = int(b[1])
        return d_b - d_a
        
    def __add__(self, other):
        new = Brain(self.in_size, self.out_size, self.hidden_depth)
        
        # New Genome
        g = {}
        for gene in self.genome:
            g[gene.innov]=gene
            
        for gene in other.genome:
            if gene.innov not in g:
                g[gene.innov]=gene.copy()
            else:
                if rd.random()>0.5:
                    g[gene.innov]=gene.copy()
                    
        new.genome = list(g.values())
        new.grow_brain()
        return new
            